# Chapter 7: Ensemble Learning and Random Forests

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import loguniform, mode
from sklearn.datasets import fetch_openml
from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
    VotingClassifier,
)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

plt.style.use("dark_background")

1. Suppose you've trained 5 different models on the same training data and each achieves a 95% precision. You could combine them using hard voting, wherein the predicted class is given by the majority of the classifiers (or soft voting if all the classifiers have a `predict_proba` method). However, this won't necessarily yield a better result if the models are very similar (i.e. all decision trees with the same hyperparameters).
2. Hard voting gives returns the majority vote of the ensemble, whereas soft voting returns the class with the highest average probability by taking an average over each of the estimators predicted class probabilities.
3. For bagging & pasting, predictors can be trained in parallel on separate servers to speed up training. A random forest is just an ensemble of decision trees trained via bagging or pasting, therefore the training of random forests can be sped up by distributing it across multiple servers. Boosting involves training weak predictors sequentially so does not benefit from parallelization. For a stacking ensemble, the predictors in each layer are independent of one another so can benefit from parallelization however each layer must be trained sequentially.
4. Out-of-bag evaluation: each predictor in a bagging ensemble is evaluated on instances it was not trained on. This makes it possible to have a fairly unbiased evaluation of the ensemble without the need for an additional validation set.
5. Extra-trees ensembles don't find the optimal splits, instead splits are made that meet a random threshold. This makes them much faster to train and the randomness introduces an element of regularization.
6. If an AdaBoost ensemble underfits the training data, increase the number of estimators or reduce the regularization hyperparameters of the base estimator. You can also try slightly increasing the learning rate.
7. If a gradient boosting ensemble overfits the training data you should decrease the learning rate. You could also try early stopping (i.e. using fewer predictors).

*bagging vs. pasting* - both bagging and pasting allow training instances to be sampled several times across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.


## 8. Voting Classification for MNIST

In [ ]:
mnist = fetch_openml("mnist_784", as_frame=False, parser="auto")
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10_000)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10_000)


def plot_digit(image_data):
    plt.imshow(image_data.reshape(28, 28), cmap="binary_r")
    plt.axis("off")


plt.figure()
for idx, image_data in enumerate(X_train[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)

plt.show()

print(
    f"X_train.shape: {X_train.shape}",
    f"X_val.shape: {X_val.shape}",
    f"X_test.shape: {X_test.shape}",
    sep="\n",
)

### 8.a. KNeighborsClassifier

In [ ]:
knn_classifier = GridSearchCV(
    KNeighborsClassifier(
        p=1,
        weights="distance",
        algorithm="ball_tree",
        n_jobs=-1,
    ),
    {"n_neighbors": [3, 4, 5, 6]},
    cv=3,
    n_jobs=-1,
    scoring="accuracy",
)
knn_classifier.fit(X_train[:3_000], y_train[:3_000])

In [ ]:
model_scores = {"model": [], "val_accuracy": [], "y_val_pred": []}


def eval_grid_search(grid_search_model, refit=False):
    print(
        f"Best params: {grid_search_model.best_params_}",
        f"Best score: {grid_search_model.best_score_}",
        sep="\n",
    )
    y_val_pred = grid_search_model.predict(X_val)
    score = accuracy_score(y_val, y_val_pred)
    print(
        f"Score on validation set: {score}",
        sep="\n",
    )
    if refit:
        print("Refitting on whole training set")
        grid_search_model.best_estimator_.fit(X_train, y_train)
        y_val_pred = grid_search_model.predict(X_val)
        score = accuracy_score(y_val, y_val_pred)
        print(f"Score on validation set: {score}")

    model_scores["model"].append(grid_search_model.best_estimator_.__class__.__name__)
    model_scores["val_accuracy"].append(score)
    model_scores["y_val_pred"].append(y_val_pred)


eval_grid_search(knn_classifier)

### 8.b. SVC

In [ ]:
svm_classifier = RandomizedSearchCV(
    make_pipeline(MinMaxScaler(), SVC()),
    {"svc__C": loguniform(0.01, 100), "svc__gamma": loguniform(0.0001, 1)},
    n_iter=20,
    cv=3,
    n_jobs=-1,
    scoring="accuracy",
)
svm_classifier.fit(X_train[:3_000], y_train[:3_000])

In [ ]:
eval_grid_search(svm_classifier)

### 8.c. RandomForestClassifier

In [ ]:
random_forest_classifier = RandomizedSearchCV(
    RandomForestClassifier(n_jobs=-1),
    {
        "n_estimators": [100 * _ for _ in range(1, 21)],
        "max_features": loguniform(0.01, 0.05),
        "max_depth": range(5, 30),
    },
    n_iter=20,
    cv=3,
    n_jobs=-1,
    scoring="accuracy",
)
random_forest_classifier.fit(X_train[:3_000], y_train[:3_000])

In [ ]:
eval_grid_search(random_forest_classifier, refit=True)

### 8.d. ExtraTreesClassifier

In [ ]:
extra_tree_classifier = RandomizedSearchCV(
    ExtraTreesClassifier(n_jobs=-1),
    {
        "n_estimators": [100 * _ for _ in range(1, 21)],
        "max_features": loguniform(0.01, 0.05),
        "max_depth": range(5, 30),
    },
    n_iter=20,
    cv=3,
    n_jobs=-1,
    scoring="accuracy",
)
extra_tree_classifier.fit(X_train[:3_000], y_train[:3_000])

In [ ]:
eval_grid_search(extra_tree_classifier, refit=True)

In [ ]:
pd.DataFrame(model_scores)

In [ ]:
votes = np.vstack(model_scores["y_val_pred"]).T
y_val_pred = mode(votes.astype("int"), axis=1).mode.astype("str")
accuracy_score(y_val, y_val_pred)

## 9. Stacking Classification for MNIST

In [ ]:
blender = RandomizedSearchCV(
    make_pipeline(MinMaxScaler(), SVC()),
    {"svc__C": loguniform(0.01, 100), "svc__gamma": loguniform(0.0001, 1)},
    n_iter=20,
    cv=3,
    n_jobs=-1,
    scoring="accuracy",
    verbose=3,
)

blender.fit(votes.astype("int"), y_val)

In [ ]:
blender.best_score_